In [1]:
sig_mod = 0
insig_mod = 0
sig_exac = 0
insig_exac = 0
dunno = 0

for i in range(0,len(ttest_pvalue[:,0])-1): #longitude
    for j in range(0,len(ttest_pvalue[0,:])-1): #latitude
        #print(ttest_pvalue[i,j])
        if land_moderated[i,j]-land_exacerbated[i,j] > 0 and ttest_pvalue[i,j] < 0.05:
            sig_mod = sig_mod + land_moderated[i,j]
            insig_exac = insig_exac + land_exacerbated[i,j]
        elif land_moderated[i,j]-land_exacerbated[i,j] > 0 and ttest_pvalue[i,j] >= 0.05:
            insig_mod = insig_mod + land_moderated[i,j]
            insig_exac = insig_exac + land_exacerbated[i,j]
        elif land_moderated[i,j]-land_exacerbated[i,j] < 0 and ttest_pvalue[i,j] < 0.05:
            sig_exac = sig_exac + land_exacerbated[i,j]
            insig_mod = insig_mod + land_moderated[i,j]
        elif land_moderated[i,j]-land_exacerbated[i,j] < 0 and ttest_pvalue[i,j] <= 0.05:
            insig_exac = insig_exac + land_exacerbated[i,j]
            insig_mod = insig_mod + land_moderated[i,j]

NameError: name 'ttest_pvalue' is not defined

In [13]:
# Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
# NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
# search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.

# Let's define a data array for the area weight of the gridcells
da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
# land area = gricell area * land fraction ([0-100] * 0.01)
da_land_area = ds_area['areacella'] * ds_land['sftlf'] * 0.01 
# Now let's create a land area weighting
da_land_weight = da_land_area / da_land_area.sum()

data_dir='/home/users/quigley/Projects/geoengineering_summer'

# Set variables that are common to all experiments.
model='UKESM1-0-LL'
centre='MOHC' 
    
if var == "evspsblpot":
    domain = "Emon"
else:
    domain='Amon' # pr is an Amon variable.
grid='gn'
season='ANN'

# specify a list of runs
runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']

# Load G6sulfur ensemble-mean datasets
exp='G6sulfur'
project='GeoMIP'
dates=['2070-01-01','2100-01-01']
ds_G6sulfur_mean, ds_G6sulfur_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

# Load ssp585 ensemble-mean datasets
project='ScenarioMIP'
exp='ssp585'
dates=['2070-01-01','2100-01-01']
ds_ssp585_mean, ds_ssp585_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

# Load ssp585 ensemble-mean datasets
project='CMIP'
exp='historical'
dates=['1960-01-01','1990-01-01']
ds_historical_mean, ds_historical_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.
 
# Call my better_worse_off function, passing in the data_arrays for the variable of interest rather than the full datasets.
better, worse, dunno = better_worse_off(ds_G6sulfur_mean[var], ds_G6sulfur_std[var], ds_ssp585_mean[var], ds_ssp585_std[var], ds_historical_mean[var], ds_historical_std[var], 90, 0.05)

#Calculating the moderated/exacerbated arrays before determining significance
moderated = (100*(better + 0) * da_weight).values
exacerbated = (100*(worse + 0) * da_weight).values
land_moderated = (100*(better + 0) * da_land_weight).values
land_exacerbated = (100*(worse + 0) * da_land_weight).values

loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN


In [32]:
def calculate(var, metric):
    control_mean=global_mean(var,'historical',['1960-01-01','1990-01-01'],'CMIP').values
    geo_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values  
    solar_mean=global_mean(var,'G6solar',['2070-01-01','2100-01-01'],'GeoMIP').values
    sulfur_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values
    warming_mean=global_mean(var,"ssp585",['2070-01-01','2100-01-01'],"ScenarioMIP").values
    
    geo_anom = geo_mean - control_mean
    sulfur_solar_anom = sulfur_mean - solar_mean
    geo_warming_anom = geo_mean - warming_mean
    warming_anom = warming_mean - control_mean
    
    if metric == "global_mean_anomaly_control":
        return geo_anom
    elif metric == "global_mean_anomaly_solar":
        return sulfur_solar_anom
    elif metric == "global_mean_anomaly_warming":
        return geo_warming_anom
    elif metric == "moderated_exacerbated":
        if geo_anom < warming_anom:
            return "moderated"
        elif warming_anom < geo_anom:
            return "exacerbated"
        else:
            return "N/A"
    elif metric == "global_moderated":
        global_moderated, global_exacerbated, land_moderated, land_exacerbated = frac_moderated_exacerbated(var)
        return global_moderated
    elif metric == "global_exacerbated":
        global_moderated, global_exacerbated, land_moderated, land_exacerbated = frac_moderated_exacerbated(var)
        return global_exacerbated
    elif metric == "land_moderated":
        global_moderated, global_exacerbated, land_moderated, land_exacerbated = frac_moderated_exacerbated(var)
        return land_moderated
    elif metric == "land_exacerbated":
        global_moderated, global_exacerbated, land_moderated, land_exacerbated = frac_moderated_exacerbated(var)
        return land_exacerbated
    else:
        return 0